In [ ]:
import math
import torch.optim as optim
import sys
#sys.path.append('../')
from BayesBackpropagation_OG import *
import matplotlib.pyplot as plt

In [ ]:
#Declare Network

#Hyperparameter setting

SAMPLES = 5
TEST_SAMPLES = 10
BATCH_SIZE = 200
NUM_BATCHES = 10
TEST_BATCH_SIZE = 75
CLASSES = 1
PI = 0.25
SIGMA_1 = torch.FloatTensor([math.exp(-0)])
SIGMA_2 = torch.FloatTensor([math.exp(-6)])

net = BayesianNetwork(inputSize = 1,\
                        CLASSES = CLASSES, \
                        layers=np.array([16,16, 16]), \
                        activations = np.array(['relu', 'relu','relu','none']), \
                        SAMPLES = SAMPLES, \
                        BATCH_SIZE = BATCH_SIZE,\
                        NUM_BATCHES = NUM_BATCHES,\
                        hasScalarMixturePrior = True,\
                        PI = PI,\
                        SIGMA_1 = SIGMA_1,\
                        SIGMA_2 = SIGMA_2,\
                        GOOGLE_INIT= False)
    


modelpath = "Models/Regression_BBB_cont_homo_10000.pth"

net.load_state_dict(torch.load(modelpath, map_location='cpu'))

In [ ]:
class Net(torch.nn.Module):
        def __init__(self, n_feature, n_hidden, n_output):
            super(Net, self).__init__()
            self.l1 = torch.nn.Linear(n_feature, n_hidden)   
            self.l2 =  torch.nn.Linear(n_hidden, n_hidden)   
            self.l3 =  torch.nn.Linear(n_hidden, n_hidden)  
            self.predict = torch.nn.Linear(n_hidden, n_output)   

        def forward(self, x):
            x = F.relu(self.l1(x))      
            x = F.relu(self.l2(x))      
            x = F.relu(self.l3(x))      
            x = self.predict(x)         
            return x

net_NN = Net(n_feature=1, n_hidden=16, n_output=1)
optimizer = torch.optim.SGD(net.parameters(), lr=0.2)

modelpath = "Models/new_data_range/Regression_NN_cont_homo.pth"

net_NN.load_state_dict(torch.load(modelpath, map_location='cpu'))

In [ ]:
# Data Generation 

# continuous data range
x = np.random.uniform(-0.5, 0.5, size=(NUM_BATCHES,BATCH_SIZE))
x_test = np.linspace(-1, 1,TEST_BATCH_SIZE)

# split cluster
# un-comment this to investigate split cluster data range
'''
x = np.random.uniform(-0.5, 0.5, size=(NUM_BATCHES,BATCH_SIZE))
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        if x[i][j]>0:
            x[i][j] += 0.5

x_test = np.linspace(-1, 1.5, 75)

'''

noise = np.random.normal(0, 0.02, size=(NUM_BATCHES,BATCH_SIZE)) #metric as mentioned in the paper
def noise_model(x):
    return 0.45*(x+0.5)**2

# HOMOSKEDASTIC REGRESSION from BLUNDELL15
y = x + 0.3*np.sin(2*np.pi*(x+noise)) + 0.3*np.sin(4*np.pi*(x+noise)) + noise
y_test = x_test + 0.3*np.sin(2*np.pi*x_test) + 0.3*np.sin(4*np.pi*x_test)

# HETEROSKEDASTIC REGRESSION
#y_test = -(x_test+0.5)*np.sin(3 * np.pi *x_test)
#y = -(x+0.5)*np.sin(3 * np.pi *x) + np.random.normal(0, noise_model(x))

Var = lambda x, dtype=torch.FloatTensor: Variable(torch.from_numpy(x).type(dtype)) #converting data to tensor
X = Var(x)
Y = Var(y)
X_test = Var(x_test)


In [ ]:
#Testing BBB
outputs = torch.zeros(TEST_SAMPLES+1, TEST_BATCH_SIZE, CLASSES)
for i in range(TEST_SAMPLES):
    outputs[i] = net.forward(X_test)
outputs[TEST_SAMPLES] = net.forward(X_test)
pred_mean = outputs.mean(0).data.cpu().numpy().squeeze(1) #Compute mean prediction
pred_std = outputs.std(0).data.cpu().numpy().squeeze(1) #Compute standard deviation of prediction for each data point


In [ ]:
#Testing NN

x = x.flatten()
X = Var(x)
X = torch.unsqueeze(X,1)
    
y = y.flatten()
Y = Var(y)
Y = torch.unsqueeze(Y,1)
X_test_NN = Var(x_test)
X_test_NN = torch.unsqueeze(X_test_NN,1)


prediction = net_NN(X_test_NN)

In [ ]:

#Visualization BB
plt.fill_between(x_test, pred_mean - 2 * pred_std, pred_mean + 2 * pred_std,
                        color='cornflowerblue', alpha=.5, label='+/- 2 std')
plt.scatter(x, y, s = 1,c='black', label='target', alpha = 0.2)
plt.ylim([-4, 4])
#plt.ylabel("y")
#plt.xlabel("x")
plt.plot(x_test, pred_mean, c='red', label='Prediction')
plt.plot(x_test, y_test, c='grey', label='Ground Truth')
#plt.title("BBB Mixture")
#plt.legend()
plt.tight_layout()
#plt.savefig('Results/new_data_range/Regression_BBB_base_paper9_1000.png')
#plt.savefig('Results/Regression_BBB_128_3_wu.eps', format='eps', dpi=1000)
#plt.clf()
plt.show()


In [ ]:
#Visualization NN
plt.scatter(x, y,s=1.5, c='black', label='target', alpha = 0.5)
plt.plot(x_test, prediction.detach().numpy(), c='red', label='Prediction')
plt.plot(x_test, y_test, c='grey', label='Ground Truth')
plt.ylim([-3, 3])
#plt.title("Neural Network")
#plt.legend()
plt.tight_layout()
plt.savefig('Results/new_data_range/Regression_NN_base_paper9_2000.png')
plt.show()

#plt.savefig('Results/Regression_NN_128_3_wu.eps', format='eps', dpi=1000)
#plt.clf()